In [1]:
import pandas as pd
import glob
import os
import re
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
#смотрим директорию 
os.getcwd()

'/Users/denisslobodyanyuk'

####  Функция для кондиционирования и электроснабжения

In [3]:
# на вход получает список путей к файлам, условие для аждой категории, ИДП 1 или ИДП 2
def get_total_data_per_condition(xl_files_by_idp, condition, category):
   

    result_frame = pd.DataFrame()
    
    
    #цикл для создания одной таблицы
    for file in xl_files_by_idp:
        
        
        #проверяем на условие
        if condition in file:
            
            #блоки из определённых файлов
            if file == '4.2.5_НЭ_Ресник Лапово_ИДП1.xls':
                
                data = pd.read_excel(file)
                data = data.loc[11:240]
                
            elif file == '4.2.7_НЭ_Ниш_ИДП1.xls':
                
                data = pd.read_excel(file)
                data = pd.concat([data.loc[296:388], data.loc[392:485]])
            
            elif file == '4.2.2_КТПМ_Лапово Ниш_ИДП2.xls':
                
                data = pd.read_excel(file)
                data = pd.concat([data.loc[3514:3727], data.loc[3731:3935], data.loc[3938:4116]])
                
            else:
        
                data = pd.read_excel(file).drop(range(9), axis=0).reset_index(drop=True)
            
            
            #проверяем количество столбцов
            if len(data.columns) == 6:
                data['НИИАС'] = data['НИИАС'].fillna('unknown')
           
            elif len(data.columns) == 9:
                data = data.drop(['НИИАС', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
            
            else:
                data = data.drop(['НИИАС', 'Unnamed: 7'], axis=1)

                
            #переименуем столбцы
            data.columns = ['description_rus', 'description_ser', 'measure', 'amount', 'price_per_item', 'total']

            
            #переведем столбцы в строки
            data['description_rus'] = data['description_rus'].astype('str')
            data['description_ser'] = data['description_ser'].astype('str')
            
            #пропуски заменим на 0
            data = data.fillna(0)

            #добавим столбец с категорией ИДП 1/ИДП 2
            data['category'] = category

            #соединим таблицы
            result_frame = pd.concat([result_frame, data])
            
    
    return result_frame

#### Функция для ВОЛС

In [4]:
def get_total_data_per_condition_vols(xl_files_by_idp, condition, category):
   

    result_frame = pd.DataFrame()
    
    #цикл для создания одной таблицы
    for file in xl_files_by_idp:
        
        
        if condition in file:
        
        #удалим первые ненужные строки
            data = pd.read_excel(file).drop(range(9), axis=0).drop('НИИАС', axis=1).reset_index(drop=True)
            

                
            #переименуем столбцы
            data.columns = ['description_rus', 'measure', 'amount', 'price_per_item', 'total']

            
            data['description_rus'] = data['description_rus'].astype('str')
            
            data = data.fillna(0)

            #добавим столбец с категорией ИДП 1/ИДП 2
            data['category'] = category

            #соединим таблицы
            result_frame = pd.concat([result_frame, data])
    
    return result_frame

####  Функция для ТСОБ

In [5]:
def get_total_data_per_condition_tsob(xl_files_by_idp, condition, category):
   

    result_frame = pd.DataFrame()
    
    #цикл для создания одной таблицы
    for file in xl_files_by_idp:
        
        
        if condition in file:
        
        #удалим первые ненужные строки
            data = pd.read_excel(file).drop(range(9), axis=0).reset_index(drop=True)
            
            
            if len(data.columns) == 6:
                data = data.drop('Unnamed: 2', axis=1)
            elif len(data.columns) == 9:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
            elif len(data.columns) == 10:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
            elif len(data.columns) == 11:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], axis=1)
            elif len(data.columns) == 12:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'], axis=1)
            elif len(data.columns) == 7:
                data = data.drop(['НИИАС','Unnamed: 2'], axis=1)
            else:
                data = data.drop(['НИИАС', 'Unnamed: 2', 'Unnamed: 7'], axis=1)

                
            #переименуем столбцы
            data.columns = ['description_ser', 'measure', 'amount', 'price_per_item', 'total']

            
            data['description_ser'] = data['description_ser'].astype('str')
            
            data = data.fillna(0)

            #добавим столбец с категорией ИДП 1/ИДП 2
            data['category'] = category

            #соединим таблицы
            result_frame = pd.concat([result_frame, data])
            
    
    return result_frame

####  Функция для категоризации материалов и работ

In [6]:
def materials_or_works(data):
    
    #столбец для категорий
    data['name'] = ''


    for x in data.index:                

        work = re.search('работы$', data.loc[x]['description_rus'])
        equipment = re.search('оборудование$', data.loc[x]['description_rus'])
        
        if  ('Материалы' in data.loc[x]['description_rus']
             or 'Материјали' in data.loc[x]['description_ser']
              or 'материјали' in data.loc[x]['description_ser']):         
            
             data['name'].loc[x:] = 'материалы'


        elif ('Оборудование' in data.loc[x]['description_rus']
              or 'опрема' in data.loc[x]['description_ser']
              or 'Опрема' in data.loc[x]['description_ser']
              or equipment != None):           
            
             data['name'].loc[x:] = 'оборудование'


        elif ('Работы' in data.loc[x]['description_rus']
              or work != None
              or 'радови' in data.loc[x]['description_rus']
              or 'радови' in data.loc[x]['description_ser']
              or 'Радови' in data.loc[x]['description_ser']):           
            
             data['name'].loc[x:] = 'работы'

#### Функция для категоризации работ, где только описание на сербском 

In [7]:
def materials_or_works_only_serbian(data):
    
    data['name'] = ''


    for x in data.index:                

        
        if  ( 'Материјали' in data.loc[x]['description_ser']
              or 'материјали' in data.loc[x]['description_ser']):         
            
             data['name'].loc[x:] = 'материалы'


        elif ( 'опрема' in data.loc[x]['description_ser']
              or 'Опрема' in data.loc[x]['description_ser']):           
            
             data['name'].loc[x:] = 'оборудование'


        elif (  'радови' in data.loc[x]['description_ser']
              or 'Радови' in data.loc[x]['description_ser']):           
            
             data['name'].loc[x:] = 'работы'

## 1. Работа с файлами " ИДП 1"

In [8]:
#меняем на нужную папку
os.chdir('/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/ИДП 1')


#сохраним пути ко всем файлам
xl_files_idp_1 = glob.glob('*.xls')

 #### 1.1 Файлы 'НЕТЯГОВОЕ ЭЛЕКТРОСНАБЖЕНИЕ ИДП1'

In [9]:
idp1_electricity = get_total_data_per_condition(xl_files_idp_1, 'НЭ', 'ИДП 1').reset_index(drop=True)

materials_or_works(idp1_electricity)
                    
                    
idp1_electricity.head()        

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,Оборудование,Опрема,0,0.0,0,0.0,ИДП 1,оборудование
1,Трансформатор силовой трехфазный/однофазный 25...,"Трансформатор снаге трофазни/монофазни 25/0,4 ...",шт.,1.0,2000,2000.0,ИДП 1,оборудование
2,Предохранитель высоковольтный,Осигурач високог напона,шт.,1.0,99,99.0,ИДП 1,оборудование
3,Ограничитель перенапряжений,Ограничавач пренапона,шт.,1.0,199,199.0,ИДП 1,оборудование
4,Разъединитель,Растављач,шт.,1.0,633,633.0,ИДП 1,оборудование


####  1.2 Файлы  'ТСОБ ИДП 1'

In [10]:
idp1_tsob = get_total_data_per_condition_tsob(xl_files_idp_1, 'ТСОБ', 'ИДП 1').drop(0, axis=0).reset_index(drop=True)

materials_or_works_only_serbian(idp1_tsob)


idp1_tsob.head()

,description_ser,measure,amount,price_per_item,total,category,name
0,"Опрема, материјали",0,0,0.00,0.00,ИДП 1,материалы
1,4Mp спољна куполна IP-камера са EXIR-осветљење...,ком.,3,639.00,1917.00,ИДП 1,материалы
2,4Mp спољна цилиндрична IP-камера са EXIR-освет...,ком.,13,640.00,8320.00,ИДП 1,материалы
3,Комутатор PSW-2G6F+ (6х10/100Base-Tx RJ-45 PoE...,ком.,3,572.52,1717.56,ИДП 1,материалы
4,Трансивер (примопредајник) 1000BASE-LX/LH SFP ...,ком.,7,991.43,6940.01,ИДП 1,материалы


####  1.3 Файлы 'КОНДИЦИОНИРОВАНИЕ ИДП 1'

In [11]:
idp1_conditioning = get_total_data_per_condition(xl_files_idp_1, 'Кондиционирование', 'ИДП 1').drop(0, axis=0).reset_index(drop=True)

materials_or_works(idp1_conditioning)



idp1_conditioning.head()

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,Оборудование,Опрема,0,0,0.00,0.00,ИДП 1,оборудование
1,Комплект кондиционера SRK60ZR-S / SRC60ZR-S,Комплетан клима-уређај SRK60ZR-S / SRC60ZR-S,шт.,4,1974.50,7898.00,ИДП 1,оборудование
2,Пульт управления проводной,Жичани даљински управљач,шт.,2,164.62,329.24,ИДП 1,оборудование
3,Нагреватель дренажной трубки,Грејач дренажне цеви,шт.,4,25.56,102.24,ИДП 1,оборудование
4,Согласователь работы кондиционеров,Контролер рада клима-уређаја,шт.,1,465.50,465.50,ИДП 1,оборудование


### 2. Работа с файлами "ИДП 2"

In [12]:
#теперь делаем тоже самое для ИДП 2
os.chdir('/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/ИДП 2')


xl_files_idp_2 = glob.glob('*.xls')
xl_files_idp_2 = xl_files_idp_2 + glob.glob('*.xlsx')

#### 2.1  Файлы 'Электроснабжение ИДП 2'

In [13]:
idp2_electricity = get_total_data_per_condition(xl_files_idp_2, 'ТП', 'ИДП 2').reset_index(drop=True) 

materials_or_works(idp2_electricity)

        
idp2_electricity.head()

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,unknown,Опрема,0,0.0,0,0.0,ИДП 2,оборудование
1,unknown,Енергетски трансформатор трофазни/једнофазни ...,ком.,1.0,2000,2000.0,ИДП 2,оборудование
2,unknown,Високонапонски осигурач,ком.,1.0,99,99.0,ИДП 2,оборудование
3,unknown,Одводник пренапона,ком.,1.0,199,199.0,ИДП 2,оборудование
4,unknown,Растављач,ком.,1.0,633,633.0,ИДП 2,оборудование


#### 2.2 Файлы 'ТСОБ ИДП 2'

In [14]:
idp2_tsob = get_total_data_per_condition_tsob(xl_files_idp_2, '06.0', 'ИДП 2').drop(0, axis=0).reset_index(drop=True)

materials_or_works_only_serbian(idp2_tsob)


idp2_tsob.head()

,description_ser,measure,amount,price_per_item,total,category,name
0,"Опрема, материјали",0,0.0,0.00,0.0,ИДП 2,материалы
1,4Mp спољна куполна IP-камера са EXIR-осветљење...,ком.,1.0,639.00,639.0,ИДП 2,материалы
2,4Mp спољна цилиндрична IP-камера са EXIR-освет...,ком.,4.0,640.00,2560.0,ИДП 2,материалы
3,Комутатор PSW-2G6F+ (6х10/100Base-Tx RJ-45 Po...,ком.,1.0,572.52,572.52,ИДП 2,материалы
4,Трансивер (примопредајник) 1000BASE-LX/LH SFP ...,ком.,8.0,991.43,7931.44,ИДП 2,материалы


#### 2.3  Файлы  'Кондиционирование ИДП 2'

In [15]:
idp2_conditioning = get_total_data_per_condition(xl_files_idp_2, 'КОНДИЦИОНИРОВАНИЕ', 'ИДП 2').drop(0, axis=0).reset_index(drop=True)

materials_or_works(idp2_conditioning)

idp2_conditioning.head()

,description_rus,description_ser,measure,amount,price_per_item,total,category,name
0,Оборудование,Опрема,0,0,0.00,0.00,ИДП 2,оборудование
1,Комплект кондиционера MSH-GF50VA / MUH-GF50VA,Комплетан клима-уређај MSH-GF50VA / MUH-GF50VA,шт.,2,1010.00,2020.00,ИДП 2,оборудование
2,Комплект кондиционера MSH-GF50VA / MUH-GF50VA,Комплетан клима-уређај MSH-GF50VA / MUH-GF50VA,шт.,2,1010.00,2020.00,ИДП 2,оборудование
3,Комплект кондиционера MSH-GF60VA / MUH-GF60VA,Комплетан клима-уређај MSH-GF60VA / MUH-GF60VA,шт.,2,1197.00,2394.00,ИДП 2,оборудование
4,Пульт управления проводной,Жичани даљински управљач,шт.,6,164.62,987.72,ИДП 2,оборудование


###  3 Общие таблицы по двум ИДП

### 3.1 Электроснабжение 

In [16]:
total_electricity = pd.concat([idp1_electricity, idp2_electricity]).drop(range(2), axis=0).reset_index(drop=True)

#### 3.1.1 Сводная таблица для материалов и оборудования

In [17]:
#выбираем нужные строи и удалим таие строки ка "Всего работ"
electricity_materials = total_electricity.query('name == ["материалы", "оборудование"] & price_per_item != 0').reset_index(drop=True)

#там, где nan, там нет описания 
electricity_materials.loc[(electricity_materials.description_ser == 'nan'), 'description_ser'] = 'no description'


electricity_materials_pivot = electricity_materials.pivot_table(                        
    index = ['name', 'category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL')



electricity_materials_pivot.tail()

amount  \
name         category description_rus                            description_ser                               measure price_per_item             
оборудование ИДП 2    Щит распределительный навесной             Разводна табла са шаркама                     шт.     7.6                 34.0   
                      Щит с монтажной панелью ЩМП 600x400x210    Табла са монтажним панелом ТМП 600x400x210    шт.     71.5                 1.0   
                      Щит с монтажной панелью ЩМП-03 350x300x155 Табла са монтажним панелом ТМП-03 350x300x155 шт.     27.7                19.0   
                      Щит с монтажной панелью ЩМП-04 400x300x155 Табла са монтажним панелом ТМП-04 400x300x155 шт.     32.3                20.0   
TOTAL                                                                                                                                  153670.8   

                                                                                                                                            total  
name         category description_rus                            description_ser                               measure price_per_item              
оборудование ИДП 2    Щит распределительный навесной             Разводна табла са шаркама                     шт.     7.6                 258.40  
                      Щит с монтажной панелью ЩМП 600x400x210    Табла са монтажним панелом ТМП 600x400x210    шт.     71.5                 71.50  
                      Щит с монтажной панелью ЩМП-03 350x300x155 Табла са монтажним панелом ТМП-03 350x300x155 шт.     27.7                526.30  
                      Щит с монтажной панелью ЩМП-04 400x300x155 Табла са монтажним панелом ТМП-04 400x300x155 шт.     32.3                646.00  
TOTAL                                                                                                                                  1093098.49

#### 3.1.2 Сводная таблица для работ

In [18]:
electricity_works = total_electricity.query('name == "работы" & price_per_item != 0').reset_index(drop=True)

electricity_works.loc[(electricity_works.description_ser == 'nan'), 'description_ser'] = 'no description'

electricity_works_pivot = electricity_works.pivot_table(                        
    index = ['category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL')



electricity_works_pivot.tail()

amount  \
category description_rus                                   description_ser                             measure price_per_item            
ИДП 2    Установка опоры КТПМ                              Постављање ослонца СТС                      шт.     18.25              23.0   
         Установка стабилизатора напряжения с подключением Уградња стабилизатора напона са повезивањем шт.     28.51              19.0   
         Установка щита ЩАО с подключением                 Уградња табле ХО са прикључком              шт.     43.81              19.0   
         Установка щита ЩР1 с подключением                 Уградња табле РТ1 са прикључком             шт.     43.81              19.0   
TOTAL                                                                                                                          59918.2   

                                                                                                                                   total  
category description_rus                                   description_ser                             measure price_per_item             
ИДП 2    Установка опоры КТПМ                              Постављање ослонца СТС                      шт.     18.25              419.75  
         Установка стабилизатора напряжения с подключением Уградња стабилизатора напона са повезивањем шт.     28.51              541.69  
         Установка щита ЩАО с подключением                 Уградња табле ХО са прикључком              шт.     43.81              832.39  
         Установка щита ЩР1 с подключением                 Уградња табле РТ1 са прикључком             шт.     43.81              832.39  
TOTAL                                                                                                                          370765.40

#### 

### 3.2 ТСОБ

In [19]:
total_tsob = pd.concat([idp1_tsob, idp2_tsob]).reset_index(drop=True)

#приведем к численному значению
total_tsob.loc[(total_tsob.amount == '2 516'), 'amount'] = 2516
total_tsob.loc[(total_tsob.amount == '1 024'), 'amount'] = 1024


#изменим тип данных на нужный
total_tsob['amount'] = total_tsob['amount'].astype('float64')
total_tsob['total'] = total_tsob['total'].astype('float64')
total_tsob.query('name == "работы"').head(40)

,description_ser,measure,amount,price_per_item,total,category,name
68,Монтажни радови,0,0.000,0.00,0.00,ИДП 1,работы
69,Пробијање у зидовима од опеке округлих отвора ...,100 ком.,0.020,3754.99,75.10,ИДП 1,работы
70,Постављање металних стубова висине до 4 м са п...,100 стубова,0.060,7510.00,450.60,ИДП 1,работы
71,Постављање монтажних уложака за спајање тежине...,т,0.660,1289.20,850.87,ИДП 1,работы
72,Уградња цевовода од полиетиленских цеви са бро...,1 км цевовода,0.314,1423.14,446.87,ИДП 1,работы
73,Постављање азбестноцементних заштитних цеви пр...,100 м цеви,0.150,234.06,35.11,ИДП 1,работы
74,Пробијање у зидовима од опеке округлих отвора ...,100 ком.,0.040,3754.99,150.20,ИДП 1,работы
75,Пробијање у зидовима од опеке округлих отвора ...,100 ком.,0.040,3754.99,150.20,ИДП 1,работы
76,Херметизација пролаза код увода кабла у експло...,1 пролаз кабла,2.000,27.54,55.08,ИДП 1,работы
77,Херметизација пролаза код увода кабла у експло...,1 пролаз кабла,4.000,27.54,110.16,ИДП 1,работы


#### 3.2.1 Сводная таблица для материалов и оборудования

In [20]:
tsob_materials = total_tsob.query('name == ["материалы", "оборудование"] & price_per_item != 0').reset_index(drop=True)


tsob_materials_pivot = tsob_materials.pivot_table(                        
    index = ['category', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL')



tsob_materials_pivot.tail()

amount  \
category description_ser                                    measure price_per_item               
ИДП 2    Читач кода проксимити картица (формат картица E... ком.    90.68              105.000   
         Шина за уземљење вертикална 1000 мм / 200 А ПЗ-... ком.    31.09                3.000   
         Шина за уземљење вертикална 1000 мм / 200 А ПЗ-... ком.    31.09                2.000   
         Шина за уземљење хоризонтална/вертикална 19" 50... ком.    17.6                 2.000   
TOTAL                                                                               270615.025   

                                                                                         total  
category description_ser                                    measure price_per_item              
ИДП 2    Читач кода проксимити картица (формат картица E... ком.    90.68              9521.40  
         Шина за уземљење вертикална 1000 мм / 200 А ПЗ-... ком.    31.09                93.27  
         Шина за уземљење вертикална 1000 мм / 200 А ПЗ-... ком.    31.09                62.18  
         Шина за уземљење хоризонтална/вертикална 19" 50... ком.    17.6                 35.20  
TOTAL                                                                               7730958.56

#### 3.2.2 Сводная таблица для работ

In [21]:
tsob_works = total_tsob.query('name == "работы" & price_per_item != 0').reset_index(drop=True)

tsob_works.loc[(tsob_works.description_ser == 'nan'), 'description_ser'] = 'no description'

tsob_works_pivot = tsob_works.pivot_table(                        
    index = ['category', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL')



tsob_works_pivot.head()

amount  \
category description_ser                                    measure price_per_item           
ИДП 1    4Mp спољна куполна IP-камера са EXIR-осветљењем... ком.    50.07             16.0   
         Аутоматски прекидач једно-, дво-, трополни који... ком.    20.02           1103.0   
         Аутоматски прекидач једно-, дво-, трополни који... ком.    13.77              4.0   
         Аутоматски прекидач једно-, дво-, трополни који... ком.    13.77             20.0   
         Аутоматски прекидач једно-, дво-, трополни који... ком.    15.02            116.0   

                                                                                       total  
category description_ser                                    measure price_per_item            
ИДП 1    4Mp спољна куполна IP-камера са EXIR-осветљењем... ком.    50.07             801.12  
         Аутоматски прекидач једно-, дво-, трополни који... ком.    20.02           22082.06  
         Аутоматски прекидач једно-, дво-, трополни који... ком.    13.77              55.08  
         Аутоматски прекидач једно-, дво-, трополни који... ком.    13.77             275.40  
         Аутоматски прекидач једно-, дво-, трополни који... ком.    15.02            1742.32

### 3.3  Кондиционирование 

In [22]:
total_conditioning = pd.concat([idp1_conditioning, idp2_conditioning]).reset_index(drop=True)

#### 3.3.1 Сводная таблица для материалов и оборудования

In [23]:
# выбираем материалы и оборудование
conditioning_materials = total_conditioning.query('name == ["материалы", "оборудование"] & price_per_item != 0').reset_index(drop=True)

# сводная таблица
conditioning_materials_pivot = conditioning_materials.pivot_table(                        
    index = ['name', 'category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL')



conditioning_materials_pivot.tail()

amount  \
name         category description_rus                             description_ser                              measure price_per_item           
оборудование ИДП 2    Комплект кондиционера SRK80ZR-S / SRC80ZR-S Комплетан клима-уређај SRK80ZR-S / SRC80ZR-S шт.     1974.5              12   
                      Нагреватель дренажной трубки                Грејач дренажне цеви                         шт.     25.56               96   
                      Пульт управления проводной                  Жичани даљински управљач                     шт.     164.62              84   
                      Согласователь работы кондиционеров          Контролер рада клима-уређаја                 шт.     465.5               37   
TOTAL                                                                                                                                   13045   

                                                                                                                                           total  
name         category description_rus                             description_ser                              measure price_per_item             
оборудование ИДП 2    Комплект кондиционера SRK80ZR-S / SRC80ZR-S Комплетан клима-уређај SRK80ZR-S / SRC80ZR-S шт.     1974.5           23694.00  
                      Нагреватель дренажной трубки                Грејач дренажне цеви                         шт.     25.56             2453.76  
                      Пульт управления проводной                  Жичани даљински управљач                     шт.     164.62           13828.08  
                      Согласователь работы кондиционеров          Контролер рада клима-уређаја                 шт.     465.5            17223.50  
TOTAL                                                                                                                                  484061.41

#### 3.3.2 Сводная таблица для работ

In [24]:
#выбираем работы
conditioning_works = total_conditioning.query('name == "работы" & price_per_item != 0').reset_index(drop=True)


#сводная таблица
conditioning_works_pivot = conditioning_works.pivot_table(                        
    index = ['category', 'description_rus', 'description_ser', 'measure', 'price_per_item'],
    values = ['amount', 'total'],
    aggfunc ='sum',
    margins = True,
    margins_name = 'TOTAL')



conditioning_works_pivot.tail()

amount  \
category description_rus                                    description_ser                                    measure price_per_item           
ИДП 2    Прокладка и монтаж дренажной трубки 16мм в коробе  Полагање и монтажа дренажне цеви 16мм у каналици   м       3.0                804   
         Прокладка и монтаж кабеля NKL в коробе             Полагање и монтажа кабла NKL у каналици            м       1.5                832   
         Прокладка и монтаж трубопроводов 15.9мм х 0.09 ... Полагање и монтажа цевовода 15.9мм х 0.09 мм  у... м       7.45               790   
         Прокладка и монтаж трубопроводов 9.5мм х 0.01 м... Полагање и монтажа цевовода 9.5мм х 0.01 мм у к... м       3.73               790   
TOTAL                                                                                                                                    7555   

                                                                                                                                           total  
category description_rus                                    description_ser                                    measure price_per_item             
ИДП 2    Прокладка и монтаж дренажной трубки 16мм в коробе  Полагање и монтажа дренажне цеви 16мм у каналици   м       3.0               2412.00  
         Прокладка и монтаж кабеля NKL в коробе             Полагање и монтажа кабла NKL у каналици            м       1.5               1248.00  
         Прокладка и монтаж трубопроводов 15.9мм х 0.09 ... Полагање и монтажа цевовода 15.9мм х 0.09 мм  у... м       7.45              5885.50  
         Прокладка и монтаж трубопроводов 9.5мм х 0.01 м... Полагање и монтажа цевовода 9.5мм х 0.01 мм у к... м       3.73              2946.70  
TOTAL                                                                                                                                  112184.36

## 4. Сохранение в EXCEL

### 4.1 Кондиционирование

In [25]:
# путь в файлу xls
file_1 = '/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/сводная_кондиционирование  — копия.xlsx'

table_1 = pd.ExcelWriter(file_1, engine='xlsxwriter')

conditioning_materials_pivot.to_excel(table_1, 'материалы')
conditioning_works_pivot.to_excel(table_1, 'работы')

#ширина столбцов
sheet = table_1.sheets['материалы']
sheet.set_column('A:B', 10)
sheet.set_column('C:D', 50)
sheet.set_column('E:H', 15)

sheet = table_1.sheets['работы']
sheet.set_column('A:A', 10)
sheet.set_column('B:C', 70)
sheet.set_column('D:G', 15)

table_1.save()

###  4.2 ТСОБ

In [26]:
file_2 = '/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/сводная_ТСОБ.xlsx'

table_2 = pd.ExcelWriter(file_2, engine='xlsxwriter')

tsob_materials_pivot.to_excel(table_2, 'материалы')
tsob_works_pivot.to_excel(table_2, 'работы')

sheet = table_2.sheets['материалы']
#sheet.set_column('A:B', 10)
#sheet.set_column('C:C', 65)
#sheet.set_column('D:G', 15)

table_2.save()

## 4.3 Элетроснабжение

In [27]:
# путь в файлу xls
file_3 = '/Users/denisslobodyanyuk/Desktop/Сметы ИДП 1 и ИДП 2/сводная_электроснабжение.xlsx'

table_3 = pd.ExcelWriter(file_3, engine='xlsxwriter')

electricity_materials_pivot.to_excel(table_3, 'материалы')
electricity_works_pivot.to_excel(table_3, 'работы')

#ширина столбцов
sheet = table_3.sheets['материалы']
sheet.set_column('A:B', 10)
sheet.set_column('C:D', 50)
sheet.set_column('E:H', 15)

sheet = table_3.sheets['работы']
sheet.set_column('A:A', 10)
sheet.set_column('B:C', 70)
sheet.set_column('D:G', 15)

table_3.save()